# 防止过拟合的练习---正则化（weight）

In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('../MNIST_data', one_hot=True)

/Users/liuyanfang/anaconda3/envs/tensorflow1.4/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


Extracting ../MNIST_data/train-images-idx3-ubyte.gz
Extracting ../MNIST_data/train-labels-idx1-ubyte.gz
Extracting ../MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ../MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

In [3]:
regularizer = tf.contrib.layers.l2_regularizer(0.0003)
W1 = tf.get_variable(name="W1", regularizer=regularizer, initializer=tf.truncated_normal([784, 512], mean=0., stddev=0.1))
b1 = tf.Variable(tf.zeros([512]))
layer1 = tf.nn.tanh(tf.matmul(x, W1) + b1)

W2 = tf.Variable(tf.truncated_normal([512, 256], mean=0., stddev=0.1))
b2 = tf.Variable(tf.zeros([256]))
layer2 = tf.nn.tanh(tf.matmul(layer1, W2) + b2 )

W3 = tf.Variable(tf.truncated_normal([256, 128], mean=0., stddev=0.1))
b3 = tf.Variable(tf.zeros([128]))
layer3 = tf.nn.tanh(tf.matmul(layer2, W3) + b3 )

W = tf.get_variable(name="W",regularizer=regularizer, initializer=tf.truncated_normal([128, 10], mean=0., stddev=0.1))
b = tf.Variable(tf.zeros([10]))
y = tf.matmul(layer3, W) + b

w_pic = tf.matmul(tf.matmul(tf.matmul(W1,W2),W3),W)

tf.add_to_collection：把变量放入一个集合，把很多变量变成一个列表


调用tf.contrib.layers.apply_regularization来生成损失函数的L2正则化项re，所传第一个参数为上面生成的正则化方法，第二个参数为none时默认值为tf.GraphKeys.WEIGHTS



In [4]:
tf.add_to_collection(tf.GraphKeys.WEIGHTS, W1)
tf.add_to_collection(tf.GraphKeys.WEIGHTS, W)
re = tf.contrib.layers.apply_regularization(regularizer)

In [5]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y)+re)

In [6]:
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

In [7]:
softmax_result = tf.nn.softmax(y)
pred = tf.argmax(softmax_result, axis=1)
true = tf.argmax(y_, axis=1)
accuracy = tf.reduce_mean(tf.cast(tf.equal(pred, true), tf.float32))

In [8]:
sess = tf.InteractiveSession()
init = tf.global_variables_initializer()
sess.run(init)

In [9]:
for i in range(5000):
    batch = mnist.train.next_batch(128)
    _, loss_, acu  = sess.run([train_step, loss, accuracy], {x: batch[0], y_: batch[1]})
    if i % 100 == 0:
        print("Step:{:>5}, Loss:{:>7.4f}, Accuracy:{:>7.2%}".format(i, loss_, acu))
loss_, acu = sess.run([loss, accuracy], {x: mnist.test.images, 
                                        y_: mnist.test.labels})
qz = sess.run(w_pic)
print("Training Finished, Loss:{:>7.4f}, Accuracy:{:>7.2%}".format(loss_, acu))

Step:    0, Loss: 2.9145, Accuracy: 10.16%
Step:  100, Loss: 0.7112, Accuracy: 89.84%
Step:  200, Loss: 0.6621, Accuracy: 94.53%
Step:  300, Loss: 0.5837, Accuracy: 93.75%
Step:  400, Loss: 0.6120, Accuracy: 92.97%
Step:  500, Loss: 0.4773, Accuracy: 97.66%
Step:  600, Loss: 0.4811, Accuracy: 98.44%
Step:  700, Loss: 0.5170, Accuracy: 96.88%
Step:  800, Loss: 0.4214, Accuracy: 99.22%
Step:  900, Loss: 0.4247, Accuracy: 97.66%
Step: 1000, Loss: 0.3979, Accuracy: 97.66%
Step: 1100, Loss: 0.4291, Accuracy: 97.66%
Step: 1200, Loss: 0.3928, Accuracy: 97.66%
Step: 1300, Loss: 0.4038, Accuracy: 98.44%
Step: 1400, Loss: 0.3560, Accuracy: 98.44%
Step: 1500, Loss: 0.3774, Accuracy: 98.44%
Step: 1600, Loss: 0.3993, Accuracy: 96.88%
Step: 1700, Loss: 0.3453, Accuracy: 98.44%
Step: 1800, Loss: 0.3011, Accuracy:100.00%
Step: 1900, Loss: 0.3602, Accuracy: 96.09%
Step: 2000, Loss: 0.3173, Accuracy: 98.44%
Step: 2100, Loss: 0.2941, Accuracy:100.00%
Step: 2200, Loss: 0.2781, Accuracy:100.00%
Step: 2300,

参考网址：https://www.cnblogs.com/linyuanzhou/p/6923607.html
    https://zhuanlan.zhihu.com/p/29297934

In [10]:
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
W_value = qz[:,0]
Min = np.min(W_value)
Max = np.max(W_value)
for i in range(len(W_value)):
    W_value[i] = int((W_value[i] - Min) / (Max - Min) * 255.)
im = Image.fromarray(np.reshape(W_value, (28, 28)).astype('uint8'))
im.show()
im.save('L2.jpg')

In [9]:
# L1 正则化
for i in range(5000):
    batch = mnist.train.next_batch(128)
    _, loss_, acu  = sess.run([train_step, loss, accuracy], {x: batch[0], y_: batch[1]})
    if i % 100 == 0:
        print("Step:{:>5}, Loss:{:>7.4f}, Accuracy:{:>7.2%}".format(i, loss_, acu))
loss_, acu = sess.run([loss, accuracy], {x: mnist.test.images, 
                                        y_: mnist.test.labels})
qz = sess.run(w_pic)
print("Training Finished, Loss:{:>7.4f}, Accuracy:{:>7.2%}".format(loss_, acu))


Step:    0, Loss:11.0428, Accuracy: 12.50%
Step:  100, Loss: 7.3247, Accuracy: 92.97%
Step:  200, Loss: 5.8761, Accuracy: 94.53%
Step:  300, Loss: 4.6248, Accuracy: 93.75%
Step:  400, Loss: 3.5541, Accuracy: 94.53%
Step:  500, Loss: 2.7260, Accuracy: 93.75%
Step:  600, Loss: 2.0635, Accuracy: 98.44%
Step:  700, Loss: 1.5486, Accuracy: 95.31%
Step:  800, Loss: 1.1803, Accuracy: 96.88%
Step:  900, Loss: 1.1321, Accuracy: 89.84%
Step: 1000, Loss:10.5767, Accuracy: 17.97%
Step: 1100, Loss: 1.4677, Accuracy: 78.12%
Step: 1200, Loss: 0.8898, Accuracy: 89.84%
Step: 1300, Loss: 0.5123, Accuracy: 97.66%
Step: 1400, Loss: 0.6155, Accuracy: 91.41%
Step: 1500, Loss: 0.5108, Accuracy: 96.88%
Step: 1600, Loss: 0.5107, Accuracy: 95.31%
Step: 1700, Loss: 0.4372, Accuracy: 96.09%
Step: 1800, Loss: 0.4857, Accuracy: 93.75%
Step: 1900, Loss: 0.3960, Accuracy: 99.22%
Step: 2000, Loss: 0.4860, Accuracy: 96.09%
Step: 2100, Loss: 0.5326, Accuracy: 91.41%
Step: 2200, Loss: 1.3252, Accuracy: 68.75%
Step: 2300,

In [15]:
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
W_value = qz[:,0]
Min = np.min(W_value)
Max = np.max(W_value)
for i in range(len(W_value)):
    W_value[i] = int((W_value[i] - Min) / (Max - Min) * 255.)
im = Image.fromarray(np.reshape(W_value, (28, 28)).astype('uint8'))
im.show()
im.save('L1.jpg')